In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Markdown, Latex, HTML

display(HTML("""
<style>
.widget-container {
    display: flex;
    align-items: center;
    text-align: center;
}

 .output {
/*font-family: "Helvetica Neue", Arial, Helvetica, Geneva, sans-serif;*/
font-family: "Charis SIL", serif; /* Make non-code text serif. */
line-height: 145%; /* added for some line spacing of text. */
width: 105ex; /* instead of 'inherit' for shorter lines */

    display: flex;
    align-items: center;
    text-align: center;
}

</style>
"""))


def compound(principal, interest_rate, number_of_compounds_yearly, years):
    return principal*(1+interest_rate/number_of_compounds_yearly)**(years*number_of_compounds_yearly)

def reverse_compound(amount, interest_rate, years):
    return amount*(interest_rate+1)**(-years)

## calculate the monthly payment for a fixed loan
def get_monthly_payment_fixed(interest_rate, years, principal):
    months = years * 12
    interest_rate = (interest_rate/12)
    monthly_payment = interest_rate / (1-pow(1+interest_rate,-months))*principal
    return monthly_payment


def subtract_from_principal(amount, principal):
    if amount > principal:
        return amount-principal, 0
    else:
        return 0, principal-amount
        
def project_mortgage(interest_rate_fixed, interest_rate_variable, repayment_term,
                       contribution, inflation, fixed_amount, variable_amount, fixed_term, contribute_maximum, years_to_model):
    interest_rate_fixed = interest_rate_fixed / 100
    interest_rate_variable = interest_rate_variable / 100
    inflation = inflation / 100
    

    p_fixed = fixed_amount
    p_variable = variable_amount

    interests = {'fixed':[], 'variable':[]}
    contributions = {'fixed':[], 'variable':[]}
    principals = {'fixed':[], 'variable':[]}
    excess_funds = []
    
    monthly_payment_fixed = get_monthly_payment_fixed(interest_rate_fixed,repayment_term,p_fixed)
    yearly_payment_fixed = monthly_payment_fixed*12
    yearly_available_funds = contribution*12
    
    
    for year in range(years_to_model):         
        ## roll fixed into variable
        if year == fixed_term:
            p_variable += p_fixed
            p_fixed = 0
            monthly_payment_fixed = 0
            yearly_payment_fixed = 0
        
        ## fixed component
        interest_fixed = 0
        if year < fixed_term:            
            p_fixed = compound(p_fixed,interest_rate_fixed,4,1)
            interest_fixed = p_fixed*interest_rate_fixed
        
        p_variable = compound(p_variable,interest_rate_variable,4,1)
        interest_variable = p_variable*interest_rate_variable 
        
        ## calculate how much is left on the fixed portion
        fixed_remainder, p_fixed = subtract_from_principal(yearly_payment_fixed, p_fixed)
        yearly_payment_fixed -= fixed_remainder
        
        ## figure out how much is contributed towards the variable
        minimum_payment_variable = get_monthly_payment_fixed(interest_rate_variable,repayment_term-year,p_variable)
        variable_contribution = min(minimum_payment_variable*12, p_variable)
        
        ## calculate excess funds for the year
        left_over_funds = yearly_available_funds-yearly_payment_fixed+fixed_remainder-variable_contribution
        
        if contribute_maximum == True:
            variable_contribution += left_over_funds
            left_over_funds = 0
            
        
        variable_remainder, p_variable = subtract_from_principal(variable_contribution, p_variable)
        #log(f"vc {variable_contribution} with p_v {p_variable}")
        #log(f"adding {variable_remainder} to {left_over_funds}")
        variable_contribution -= variable_remainder
        left_over_funds += variable_remainder
        
        interests['fixed'].append(interest_fixed)
        interests['variable'].append(interest_variable)
        contributions["fixed"].append(yearly_payment_fixed)
        contributions["variable"].append(variable_contribution)
        principals["fixed"].append(p_fixed)
        principals["variable"].append(p_variable)
        excess_funds.append(left_over_funds)

        if p_fixed <= 0 and p_variable <= 0:
            years_left = years_to_model - year - 1
            interests['fixed'] += [0] * years_left
            interests['variable'] += [0] * years_left
            contributions["fixed"]+= [0] * years_left
            contributions["variable"] += [0] * years_left
            principals["fixed"] += [0] * years_left
            principals["variable"] += [0] * years_left
            break
            
    
    scenario_results = {'years': year,
                       'interests': interests,
                        'contributions': contributions,
                        'principals': principals,
                        'excess_funds': excess_funds
                       }
    
    log("Finished running simulation")
    return scenario_results


def log(s):
    display(Markdown(s))
    
def scenario_report(scenario_results, scenario_name, years_to_model, investment_contribution, roi):
    log(f"Generating report for {scenario_name}")
    years = scenario_results["years"]
    months = years * 12
    
    total_interest = sum(scenario_results["interests"]["fixed"])+sum(scenario_results["interests"]["variable"])
    monthly_interest = total_interest/(months)
    
    mortgage_contributions = sum(scenario_results["contributions"]['fixed']+scenario_results["contributions"]['variable'])
    
    log(f"Total interest paid: \${total_interest:,.2f}")
    log(f"Monthly interest paid \${monthly_interest:,.2f}")
    log(f"Total contributions made: \${mortgage_contributions:,.2f}")
    
    #print(scenario_results["principals"]["fixed"])
    #print(scenario_results["contributions"]["fixed"])
    #log(" - ".join([f"{e:,.0f}" for e in scenario_results["principals"]["fixed"]]))
    #log(" - ".join([f"{e:,.0f}" for e in scenario_results["contributions"]["fixed"]]))
    #print("")
    
    #log(" - ".join([f"{e:,.0f}" for e in scenario_results["principals"]["variable"]]))
    #log(" - ".join([f"{e:,.0f}" for e in scenario_results["contributions"]["variable"]]))
    
    investments = []
    investment_balance = 0

    excess = scenario_results['excess_funds']
    for i in range(years_to_model):
        investment_balance = compound(investment_balance,roi/100,1,1)
        if i < years:
            investment_balance += excess[i]
            #log(f"{excess[i]:,.2f}")
        else:
            investment_balance += investment_contribution*12
            
        
        investments.append(investment_balance)
        
    #inf_adjusted_ret = int(reverse_compound(bank_balance,inflation,len(ps)))
    #inf_adjusted_cont = reverse_compound(sum([c * 12 for c in contributions]),inflation,len(ps))
    #percent_increase_real = int(inf_adjusted_ret/inf_adjusted_cont*100)-100
    log(f"Final Investments Value: \${investment_balance:,.2f}")
    #display(("Equivalent to ~${:,} today and {:,}% return in real terms".format(
    #        inf_adjusted_ret,percent_increase_real)))
    return investments

    
def plot_scenario(scenario_results,investments, years_to_model):
    ind = np.arange(years_to_model)    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence
    

    contributions = map(lambda x,y: x+y,
                        scenario_results["contributions"]["variable"],
                        scenario_results["contributions"]["fixed"])
    total_debts = map(lambda x,y,l,m: x+y+l+m,
                        scenario_results["principals"]["variable"],
                        scenario_results["principals"]["fixed"],
                        scenario_results["interests"]["variable"],
                        scenario_results["interests"]["fixed"]
                     )
    
    bots_fixed1 = list(map(lambda x,y: x+y,
                        scenario_results["principals"]["variable"],
                        scenario_results["interests"]["variable"],
                     ))
    
    bots_fixed2 = list(map(lambda x,y,l: x+y+l,
                        scenario_results["principals"]["variable"],
                        scenario_results["principals"]["fixed"],
                        scenario_results["interests"]["variable"],
                     ))
    
    fig = plt.figure(figsize=(16,12),dpi=300, facecolor='w')
    
    p1 = plt.bar(ind-width/2, scenario_results["principals"]["variable"], width, edgecolor='black',linewidth=1)
    p2 = plt.bar(ind-width/2, scenario_results["interests"]["variable"], width,
                 bottom=scenario_results["principals"]["variable"], edgecolor='black',linewidth=1)
    p4 = plt.bar(ind-width/2, scenario_results["principals"]["fixed"],
                 width, bottom=bots_fixed1, edgecolor='black',linewidth=1)
    p5 = plt.bar(ind-width/2, scenario_results["interests"]["fixed"], width,
                 bottom=bots_fixed2, edgecolor='black',linewidth=1)

    p6 = plt.bar(ind+width/2, investments, width, edgecolor='black',linewidth=1)
    
    tots = list(map(lambda x,z: x-z if x-z > 0 else 0,total_debts,contributions))
    tops = list(map(lambda x,z: x-z/12 if x-z/12 > 0 else 0,total_debts,tots))

    #p3 = plt.bar(ind, tops, width,
    #            bottom=tots,fill=False, hatch="/")

    plt.ylabel('Amount in AUD')
    plt.xlabel('Year')
    plt.title('Loan Payments Over {} Years'.format(years_to_model))
    plt.xticks(np.arange(0,years_to_model),np.arange(1,years_to_model+1))
    yticks = list(np.arange(0, 400000, 50000))
    #yticks.append(interests[0]+ps[0])
    #plt.yticks(yticks)
    plt.legend((p1,p2,p4,p5), ('Principal (v)', 'Interest (v)','Principal (f)', 'Interest (f)'))
    #plt.show()
    
    

def project_house_value():
    house_value = compound(house_value,annual_growth_house,1,len(ps)) - p
    display(Markdown(" Net value of house: ${:.2f}".format(int(house_value))))
    house_value_today = reverse_compound(house_value,inflation,len(ps))
    #percent_increase_house_real = int((house_value_today-house_value_original)/(inf_adjusted_cont*100)-100)
    #display(Markdown(" Equivalent to ~${:,} today and {:,}% return in real terms".format(
    #int(house_value_today),percent_increase_house_real)))
    
def house_calculations(house_value, interest_rate_fixed, interest_rate_variable, repayment_term, roi,
                       contribution,inflation,fixed_amount, variable_amount, fixed_term, years_to_model):
        
        
    minimum_cont_scenario = project_mortgage(interest_rate_fixed, interest_rate_variable, repayment_term,
                       contribution, inflation,fixed_amount,
                                             variable_amount, fixed_term, False, years_to_model)
    
    maximum_cont_scenario = project_mortgage(interest_rate_fixed, interest_rate_variable, repayment_term,
                       contribution, inflation,fixed_amount,
                                             variable_amount, fixed_term, True, years_to_model)
    
    investments_min = scenario_report(minimum_cont_scenario, 'Minimum Contribution', years_to_model, contribution, roi)
    investments_max = scenario_report(maximum_cont_scenario, 'Maximum Contribution', years_to_model, contribution, roi)

    #print(investments_min)
    #print(investments_max)
    
    plot_scenario(minimum_cont_scenario,investments_min, years_to_model)
    plot_scenario(maximum_cont_scenario,investments_max, years_to_model)
    

style = {'description_width': 'initial', 'max-width':'100%'}
iplot = interactive(house_calculations, 
             house_value = widgets.IntSlider(value=2.6E6,min=2E6,max=3E6, step=10000,
    description='House Value', style=style),
         interest_rate_fixed = widgets.FloatSlider(value=2.3,min=2,max=10.0, step=0.01,
    description='Interest Rate Fixed', style=style),
         interest_rate_variable = widgets.FloatSlider(value=2.7,min=2,max=10.0, step=0.01,
    description='Interest Rate Variable', style=style),
         repayment_term = widgets.IntSlider(value=10,min=5,max=30, step=1,
    description='Repayment Term', style=style),
         roi = widgets.FloatSlider(value=7.5,min=0,max=15.0, step=0.1,
    description='Investment Growth Rate', continuous_update=False, style=style),
         annual_growth_house = widgets.FloatSlider(value=6,min=0,max=15.0, step=0.1,
    description='Annual House Growth', style=style),
        contribution = widgets.IntSlider(value=4000,min=2000,max=5000, step=100,
    description='Monthly Mortgage', style=style),
        inflation = widgets.FloatSlider(value=2,min=0,max=10.0, step=0.1,
    description='Inflation', style=style),
    fixed_amount = widgets.IntSlider(value=250000,min=0,max=320000, step=10000,
    description='Fixed Amount', style=style),
    variable_amount = widgets.IntSlider(value=70000,min=0,max=32000, step=10000,
    description='Variable Amount', style=style),
        fixed_term = widgets.IntSlider(value=3,min=0,max=5, step=1,
    description='Fixed Term', style=style),
    years_to_model = widgets.IntSlider(value=10,min=1,max=20, step=1,
    description='Years to Model', style=style)
    )

iplot

interactive(children=(IntSlider(value=2600000, description='House Value', max=3000000, min=2000000, step=10000…